## Add GPS Metadata capture for point, line, polygon layers

This notebook configures an existing point, line or polygon feature layer to record GPS metadata using ArcGIS Field Maps.

The notebook performs the following actions:

- Adds the necessary domains
- Adds a series of fields and associates domains
- Enables attachments if needed (line and polygon layers only)

Note: you must be the owner of the feature layer or an organization admin user

### Steps

1. Update the following variables
    - `org_url`
    - `username`    
    - `item_id`
    - `feature_layer_index`
    - `include_offset_fields`
2. Run all cells


In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# update this information prior to running
org_url = "https://www.arcgis.com"
username = "doug_nitro"
item_id = "7984f1a3089c47eaad07086b19f6fe92" # item_id of feature layer
feature_layer_index = 0 #index of layer you want to update
include_offset_fields = False # indicate if you want offset fields added (point feature layers only)

gis = GIS(org_url, username)

In [ ]:
# get the feature layer by ID
item = gis.content.get(item_id)

if (item is None):
    raise TypeError("Cannot Find item")

# check the item type is a feature service
if (item.type != 'Feature Service'):
    raise TypeError("Item is not a feature service")

feature_layer_collection = FeatureLayerCollection.fromitem(item)

feature_layer = feature_layer_collection.layers[feature_layer_index]

# get the geometry type
geometry_type = feature_layer.properties.geometryType

# make sure layer is a point, line, or polygon layer
if ((geometry_type != 'esriGeometryPolyline') and 
    (geometry_type != 'esriGeometryPolygon')and 
    (geometry_type != 'esriGeometryPoint')):
    raise TypeError("Feature layer is not a point, line, or polygon layer")

# check and enable attachments
if (geometry_type != 'esriGeometryPoint' and not feature_layer.properties.hasAttachments):    
    feature_layer.manager.update_definition({"hasAttachments": True})
    print ("Enabled attachments")

# Extract existing fields from Feature layer service definition
feature_layer_fields = feature_layer.properties.fields

# New fields which need to be added
gnss_metadata_fields = {'fields': []}

# Operations list - Add or Update GNSS Metadata fields.
operations = []

In [ ]:
# Ensure all fields are present; if not, add them to the definition

if (geometry_type != 'esriGeometryPoint'):

    # esrignss_avg_h_rms
    avg_horizontal_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_avg_h_rms']

    if not avg_horizontal_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_avg_h_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Average horizontal accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_avg_v_rms
    avg_vertical_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_avg_v_rms']

    if not avg_vertical_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_avg_v_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Average vertical accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_worst_h_rms
    worst_horizontal_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_worst_h_rms']

    if not worst_horizontal_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_worst_h_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Worst horizontal accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_worst_v_rms
    worst_vertical_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_worst_v_rms']

    if not worst_vertical_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_worst_v_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Worst vertical accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})


    # esrignss_worst_fixtype
    worst_fixtype_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_worst_fixtype']

    if worst_fixtype_field:
        # Field does exist check if the domain is set.
        if worst_fixtype_field[0]['domain'] is None:
            #if ([operation for operation in operations if operation == 'updateDefinition']):
            operations.append('updateDefinition')

            worst_fixtype_field_index = feature_layer_fields.index(worst_fixtype_field[0])
            fixtype_domain = {'type': 'codedValue',
                                'name': 'esri_fix_type_domain',
                                'codedValues': [{'name': 'Fix not valid',
                                                'code': 0},
                                                {'name': 'GPS', 'code': 1},
                                                {'name': 'Differential GPS', 'code': 2},
                                                {'name': 'RTK Fixed', 'code': 4},
                                                {'name': 'RTK Float', 'code': 5}]}
            feature_layer_fields[worst_fixtype_field_index]['domain'] = fixtype_domain

    else:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_worst_fixtype',
                                                'type': 'esriFieldTypeSmallInteger',
                                                'alias': 'Worst fix type',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': {'type': 'codedValue',
                                                        'name': 'esri_fix_type_domain',
                                                        'codedValues': [
                                                            {'name': 'Fix not valid', 'code': 0},
                                                            {'name': 'GPS', 'code': 1},
                                                            {'name': 'Differential GPS', 'code': 2},
                                                            {'name': 'RTK Fixed', 'code': 4},
                                                            {'name': 'RTK Float', 'code': 5}]},
                                                'defaultValue': None})

    # esrignss_manual_locations
    num_manual_locations_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_manual_locations']

    if not num_manual_locations_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_manual_locations',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'Number of manual locations',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})
else:
    # esrignss_positionsourcetype
    positionsourcetype_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_positionsourcetype']

    if positionsourcetype_field:
        # Field does exist; check if the domain is set
        if positionsourcetype_field[0]['domain'] is None:
            operations.append('updateDefinition')

            positionsourcetype_field_index = feature_layer_fields.index(positionsourcetype_field[0])
            positionsourcetype_domain = {'type': 'codedValue',
                                'name': 'esri_positionsourcetype_domain',
                                'codedValues': [{'name': 'Unknown',
                                                'code': 0},
                                                {'name': 'User defined', 'code': 1},
                                                {'name': 'Integrated (System) Location Provider', 'code': 2},
                                                {'name': 'External GNSS Receiver', 'code': 3},
                                                {'name': 'Network Location Provider', 'code': 4},
                                                {'name': 'Snapped', 'code': 5}]},
            feature_layer_fields[positionsourcetype_field_index]['domain'] = positionsourcetype_domain

    else:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_positionsourcetype',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'Position source type',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': {'type': 'codedValue',
                                                        'name': 'esri_positionsourcetype_domain',
                                                        'codedValues': [
                                                            {'name': 'Unknown', 'code': 0},
                                                            {'name': 'User defined', 'code': 1},
                                                            {'name': 'Integrated (System) Location Provider', 'code': 2},
                                                            {'name': 'External GNSS Receiver', 'code': 3},
                                                            {'name': 'Network Location Provider', 'code': 4},
                                                            {'name': 'Snapped', 'code': 5}]},
                                                'defaultValue': None})

    # esrignss_receiver
    receiver_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_receiver']
    
    if not receiver_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_receiver',
                                                'type': 'esriFieldTypeString',
                                                'alias': 'Receiver name',
                                                'sqlType': 'sqlTypeOther',
                                                'length': 50,
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_latitude
    latitude_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_latitude']

    if not latitude_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_latitude',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Latitude',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_longitude
    longitude_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_longitude']

    if not longitude_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_longitude',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Longitude',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_altitude
    altitude_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_altitude']

    if not altitude_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_altitude',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Altitude',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_h_rms
    horizontal_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_h_rms']

    if not horizontal_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_h_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Horizontal accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})
    # esrignss_v_rms
    vertical_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_v_rms']

    if not vertical_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_v_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Vertical accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_fixdatetime
    fix_time_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_fixdatetime']

    if not fix_time_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_fixdatetime',
                                                'type': 'esriFieldTypeDate',
                                                'alias': 'Fix time',
                                                'sqlType': 'sqlTypeOther',
                                                'length': 0,
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_fixtype
    fix_type_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_fixtype']

    if fix_type_field:
        # Field does exist; check if the domain is set
        if fix_type_field[0]['domain'] == None:
            if ([operation for operation in operations if operation == 'updateDefinition']):
                operations.append('updateDefinition')

            fix_type_field_index = feature_layer_fields.index(fix_type_field[0])
            fix_type_domain = {'type': 'codedValue',
                                'name': 'esri_fix_type_domain',
                                'codedValues': [{'name': 'Fix not valid',
                                                'code': 0},
                                                {'name': 'GPS', 'code': 1},
                                                {'name': 'Differential GPS', 'code': 2},
                                                {'name': 'RTK Fixed', 'code': 4},
                                                {'name': 'RTK Float', 'code': 5}]}
            feature_layer_fields[fix_type_field_index]['domain'] = fix_type_domain

    else:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_fixtype',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'Fix type',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': {'type': 'codedValue',
                                                        'name': 'esri_fix_type_domain',
                                                        'codedValues': [
                                                            {'name': 'Fix not valid', 'code': 0},
                                                            {'name': 'GPS', 'code': 1},
                                                            {'name': 'Differential GPS', 'code': 2},
                                                            {'name': 'RTK Fixed', 'code': 4},
                                                            {'name': 'RTK Float', 'code': 5}]},
                                                'defaultValue': None})

    # esrignss_correctionage
    correction_age_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_correctionage']

    if not correction_age_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_correctionage',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Correction age',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_stationid
    station_id_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_stationid']

    if not station_id_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_stationid',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'Station ID',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_numsats
    num_sats_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_numsats']

    if not num_sats_field: 
        gnss_metadata_fields['fields'].append({'name': 'esrignss_numsats',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'Number of satellites',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_pdop
    pdop_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_pdop']

    if not pdop_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_pdop',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'PDOP',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_hdop
    hdop_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_hdop']

    if not hdop_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_hdop',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'HDOP',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_vdop
    vdop_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_vdop']

    if not vdop_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_vdop',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'VDOP',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})
        
    # esrignss_direction
    direction_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_direction']

    if not direction_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_direction',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Direction of travel (°)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_speed
    speed_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_speed']

    if not speed_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_speed',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Speed (km/h)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrisnsr_azimuth
    azimuth_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrisnsr_azimuth']

    if not azimuth_field:
        gnss_metadata_fields['fields'].append({'name': 'esrisnsr_azimuth',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Compass reading (°)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_avg_h_rms
    average_horizontal_accuracy_field = [field for field in feature_layer_fields if
                                        field['name'].lower() == 'esrignss_avg_h_rms']

    if not average_horizontal_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_avg_h_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Average horizontal accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_avg_v_rms
    average_vertical_accuracy_field = [field for field in feature_layer_fields if
                                    field['name'].lower() == 'esrignss_avg_v_rms']

    if not average_vertical_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_avg_v_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Average vertical accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_avg_positions
    average_positions_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_avg_positions']

    if not average_positions_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_avg_positions',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'Averaged positions',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_h_stddev"
    standard_deviation_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_h_stddev']

    if not standard_deviation_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_h_stddev',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Standard deviation (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})       


In [ ]:
# Add metadata fields for laser offset workflows 
if include_offset_fields == True and geometry_type == 'esriGeometryPoint':


    # esrisnsr_offset_method
    offset_method_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrisnsr_offset_method']

    if offset_method_field:
        # Field does exist; check if the domain is set
        if offset_method_field[0]['domain'] == None:
            if ([operation for operation in operations if operation == 'updateDefinition']):
                operations.append('updateDefinition')

            offset_method_field_index = feature_layer_fields.index(offset_method_field[0])
            offset_method_domain = {'type': 'codedValue',
                                'name': 'esri_offset_method_domain',
                                'codedValues': [{'name': 'Fix not valid', 'code': 0},
                                                {'name': 'GPS', 'code': 1},
                                                {'name': 'Differential GPS', 'code': 2},
                                                {'name': 'RTK Fixed', 'code': 4},
                                                {'name': 'RTK Float', 'code': 5}]}
            feature_layer_fields[fix_type_field_index]['domain'] = offset_method_domain

    else:
        gnss_metadata_fields['fields'].append({'name': 'esrisnsr_offset_method',
                                                'type': 'esriFieldTypeString',
                                                'length': 50,
                                                'alias': 'Offset method',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': {'type': 'codedValue',
                                                        'name': 'esri_offset_method_domain',
                                                        'codedValues': [
                                                            {'name': 'Azimuth-Azimuth', 'code': 'azimuth-azimuth'},                                                                
                                                            {'name': 'Distance-Angle', 'code': 'distance-angle'},
                                                            {'name': 'Distance-Azimuth', 'code': 'distance-azimuth'},
                                                            {'name': 'Distance-Distance', 'code': 'distance-distance'},
                                                            {'name': 'Azimuth-Azimuth-Backsight', 'code': 'azimuth-azimuth-backsight'},
                                                            {'name': 'Distance-Azimuth-Backsight', 'code': 'distance-azimuth-backsight'},
                                                            {'name': 'Distance-Distance-Backsight', 'code': 'distance-distance-backsight'},
                                                            {'name': 'User-defined', 'code': 'user-defined'}]},
                                                'defaultValue': None})


    # esrisnsr_offset_device
    offset_device_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrisnsr_offset_device']

    if not offset_device_field:
        gnss_metadata_fields['fields'].append({'name': 'esrisnsr_offset_device',
                                                'type': 'esriFieldTypeString',
                                                'alias': 'Rangefinder name',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})


    # esrignss_antenna_height
    antenna_height_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_antenna_height']

    if not antenna_height_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_antenna_height',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'GNSS Antenna height (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrisnsr_offset_laser_height
    laser_height_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrisnsr_offset_laser_height']

    if not laser_height_field:
        gnss_metadata_fields['fields'].append({'name': 'esrisnsr_offset_laser_height',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Rangefinder height (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrisnsr_offset_laser_target_ht
    laser_target_height_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrisnsr_offset_laser_target_ht']

    if not laser_target_height_field:
        gnss_metadata_fields['fields'].append({'name': 'esrisnsr_offset_laser_target_ht',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Laser target height (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrisnsr_offset_mag_dec
    offset_mag_var_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrisnsr_offset_mag_var']

    if not offset_mag_var_field:
        gnss_metadata_fields['fields'].append({'name': 'esrisnsr_offset_mag_var',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Magnetic variation (°)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_latitude
    offset_cp1_latitude_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_cp1_latitude']

    if not offset_cp1_latitude_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_latitude',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 Latitude',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_longitude
    offset_cp1_longitude_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_cp1_longitude']

    if not offset_cp1_longitude_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_longitude',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 Longitude',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_altitude
    offset_cp1_altitude_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_cp1_altitude']

    if not offset_cp1_altitude_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_altitude',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 Ellipsoidal height (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_h_rms
    cp1_horizontal_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp1_h_rms']

    if not cp1_horizontal_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_h_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 Horizontal accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})
    # esrignss_cp1_v_rms
    cp1_vertical_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp1_v_rms']

    if not cp1_vertical_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_v_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 Vertical accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_fixdatetime
    cp1_fix_time_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp1_fixdatetime']

    if not cp1_fix_time_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_fixdatetime',
                                                'type': 'esriFieldTypeDate',
                                                'alias': 'CP1 Fix time',
                                                'sqlType': 'sqlTypeOther',
                                                'length': 0,
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_fixtype
    cp1_fix_type_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp1_fixtype']

    if cp1_fix_type_field:
        # Field does exist; check if the domain is set
        if fix_type_field[0]['domain'] == None:
            if ([operation for operation in operations if operation == 'updateDefinition']):
                operations.append('updateDefinition')

            fix_type_field_index = feature_layer_fields.index(fix_type_field[0])
            fix_type_domain = {'type': 'codedValue',
                                'name': 'esri_fix_type_domain',
                                'codedValues': [{'name': 'Fix not valid',
                                                'code': 0},
                                                {'name': 'GPS', 'code': 1},
                                                {'name': 'Differential GPS', 'code': 2},
                                                {'name': 'RTK Fixed', 'code': 4},
                                                {'name': 'RTK Float', 'code': 5}]}
            feature_layer_fields[fix_type_field_index]['domain'] = fix_type_domain

    else:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_fixtype',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'CP1 Fix type',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': {'type': 'codedValue',
                                                        'name': 'esri_fix_type_domain',
                                                        'codedValues': [
                                                            {'name': 'Fix not valid', 'code': 0},
                                                            {'name': 'GPS', 'code': 1},
                                                            {'name': 'Differential GPS', 'code': 2},
                                                            {'name': 'RTK Fixed', 'code': 4},
                                                            {'name': 'RTK Float', 'code': 5}]},
                                                'defaultValue': None})

    # esrignss_cp1_corr_age
    cp1_correction_age_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp1_corr_age']

    if not cp1_correction_age_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_corr_age',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 Correction age',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_stationid
    station_id_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp1_stationid']

    if not station_id_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_stationid',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'CP1 Station ID',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_numsats
    cp1_num_sats_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp1_numsats']

    if not cp1_num_sats_field: 
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_numsats',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'CP1 Number of satellites',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_pdop
    cp1_pdop_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp1_pdop']

    if not cp1_pdop_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_pdop',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 PDOP',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_hdop
    cp1_hdop_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp1_hdop']

    if not cp1_hdop_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_hdop',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 HDOP',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_vdop
    cp1_vdop_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp1_vdop']

    if not cp1_vdop_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_vdop',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 VDOP',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_avg_h_rms
    cp1_average_horizontal_accuracy_field = [field for field in feature_layer_fields if
                                        field['name'].lower() == 'esrignss_cp1_avg_h_rms']

    if not cp1_average_horizontal_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_avg_h_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 Average horizontal accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_avg_v_rms
    cp1_average_vertical_accuracy_field = [field for field in feature_layer_fields if
                                    field['name'].lower() == 'esrignss_cp1_avg_v_rms']

    if not cp1_average_vertical_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_avg_v_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 Average vertical accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_avg_positions
    cp1_average_positions_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_cp1_avg_positions']

    if not cp1_average_positions_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_avg_positions',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'CP1 Averaged positions',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp1_h_stddev
    cp1_standard_deviation_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_cp1_h_stddev']


    if not cp1_standard_deviation_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp1_h_stddev',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 Standard deviation (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrisnsr_cp1_slope_dist
    cp1_slope_distance_field = [field for field in feature_layer_fields if
                                    field['name'].lower() == 'esrisnsr_cp1_slope_dist']

    if not cp1_slope_distance_field:
        gnss_metadata_fields['fields'].append({'name': 'esrisnsr_cp1_slope_dist',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 Laser slope distance (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrisnsr_cp1_inclination
    cp1_laser_inclination_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrisnsr_cp1_inclination']

    if not cp1_laser_inclination_field:
        gnss_metadata_fields['fields'].append({'name': 'esrisnsr_cp1_inclination',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 Laser inclination (°)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrisnsr_cp1_azimuth
    cp1_standard_deviation_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrisnsr_cp1_azimuth']


    if not cp1_standard_deviation_field:
        gnss_metadata_fields['fields'].append({'name': 'esrisnsr_cp1_azimuth',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP1 Laser azimuth (°)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

# esrignss_cp2_latitude
    offset_cp2_latitude_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_cp2_latitude']

    if not offset_cp2_latitude_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_latitude',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 Latitude',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp2_longitude
    offset_cp2_longitude_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_cp2_longitude']

    if not offset_cp2_longitude_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_longitude',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 Longitude',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp2_altitude
    offset_cp2_altitude_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_cp2_altitude']

    if not offset_cp2_altitude_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_altitude',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 Ellipsoidal height (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp2_h_rms
    cp2_horizontal_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp2_h_rms']

    if not cp2_horizontal_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_h_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 Horizontal accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})
    # esrignss_cp2_v_rms
    cp2_vertical_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp2_v_rms']

    if not cp2_vertical_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_v_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 Vertical accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp2_fixdatetime
    cp2_fix_time_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp2_fixdatetime']

    if not cp2_fix_time_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_fixdatetime',
                                                'type': 'esriFieldTypeDate',
                                                'alias': 'CP2 Fix time',
                                                'sqlType': 'sqlTypeOther',
                                                'length': 0,
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp2_fixtype
    cp2_fix_type_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp2_fixtype']

    if cp2_fix_type_field:
        # Field does exist; check if the domain is set
        if fix_type_field[0]['domain'] == None:
            if ([operation for operation in operations if operation == 'updateDefinition']):
                operations.append('updateDefinition')

            fix_type_field_index = feature_layer_fields.index(fix_type_field[0])
            fix_type_domain = {'type': 'codedValue',
                                'name': 'esri_fix_type_domain',
                                'codedValues': [{'name': 'Fix not valid',
                                                'code': 0},
                                                {'name': 'GPS', 'code': 1},
                                                {'name': 'Differential GPS', 'code': 2},
                                                {'name': 'RTK Fixed', 'code': 4},
                                                {'name': 'RTK Float', 'code': 5}]}
            feature_layer_fields[fix_type_field_index]['domain'] = fix_type_domain

    else:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_fixtype',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'CP2 Fix type',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': {'type': 'codedValue',
                                                        'name': 'esri_fix_type_domain',
                                                        'codedValues': [
                                                            {'name': 'Fix not valid', 'code': 0},
                                                            {'name': 'GPS', 'code': 1},
                                                            {'name': 'Differential GPS', 'code': 2},
                                                            {'name': 'RTK Fixed', 'code': 4},
                                                            {'name': 'RTK Float', 'code': 5}]},
                                                'defaultValue': None})

    # esrignss_cp2_corr_age
    cp2_correction_age_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp2_corr_age']

    if not cp2_correction_age_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_corr_age',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 Correction age',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp2_stationid
    cp2_station_id_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp2_stationid']

    if not cp2_station_id_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_stationid',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'CP2 Station ID',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp2_numsats
    cp2_num_sats_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp2_numsats']

    if not cp2_num_sats_field: 
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_numsats',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'CP2 Number of satellites',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp2_pdop
    cp2_pdop_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp2_pdop']

    if not cp2_pdop_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_pdop',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 PDOP',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp2_hdop
    cp2_hdop_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp2_hdop']

    if not cp2_hdop_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_hdop',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 HDOP',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp2_vdop
    cp2_vdop_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_cp2_vdop']

    if not cp2_vdop_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_vdop',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 VDOP',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp2_avg_h_rms
    cp2_average_horizontal_accuracy_field = [field for field in feature_layer_fields if
                                        field['name'].lower() == 'esrignss_cp2_avg_h_rms']

    if not cp2_average_horizontal_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_avg_h_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 Average horizontal accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp2_avg_v_rms
    cp2_average_vertical_accuracy_field = [field for field in feature_layer_fields if
                                    field['name'].lower() == 'esrignss_cp2_avg_v_rms']

    if not cp2_average_vertical_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_avg_v_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 Average vertical accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp2_avg_positions
    cp2_average_positions_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_cp2_avg_positions']

    if not cp2_average_positions_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_avg_positions',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'CP2 Averaged positions',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_cp2_h_stddev
    cp2_standard_deviation_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_cp2_h_stddev']


    if not cp2_standard_deviation_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_cp2_h_stddev',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 Standard deviation (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrisnsr_cp2_slope_dist
    cp2_slope_distance_field = [field for field in feature_layer_fields if
                                    field['name'].lower() == 'esrisnsr_cp2_slope_dist']

    if not cp2_slope_distance_field:
        gnss_metadata_fields['fields'].append({'name': 'esrisnsr_cp2_slope_dist',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 Laser slope distance (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrisnsr_cp2_inclination
    cp2_laser_inclination_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrisnsr_cp2_inclination']

    if not cp2_laser_inclination_field:
        gnss_metadata_fields['fields'].append({'name': 'esrisnsr_cp2_inclination',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 Laser inclination (°)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrisnsr_cp2_azimuth
    cp2_standard_deviation_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrisnsr_cp2_azimuth']


    if not cp2_standard_deviation_field:
        gnss_metadata_fields['fields'].append({'name': 'esrisnsr_cp2_azimuth',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'CP2 Laser azimuth (°)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})


In [ ]:
# Check if AddToDefinition operation needs to be added.
initial_featurelayer_fields_count = len(feature_layer_fields)
if (len(gnss_metadata_fields[
            'fields']) + initial_featurelayer_fields_count
        ) > initial_featurelayer_fields_count:
    operations.append('addToDefinition')
    
# Check if any changes are required
if not operations:
    print("No changes to GPS metadata fields.")
else:
    # Add or update service definition
    for operation in operations:

        # Add
        if operation == 'addToDefinition':
            response = feature_layer.manager.add_to_definition(
                gnss_metadata_fields)
            print("Successfully added GPS metadata fields.")

        # Modify
        else:
            response = feature_layer.manager.update_definition(
                feature_layer_fields)
            print("Successfully updated GPS metadata fields.")

        result = response['success']

        if not result:
            print('Failed to update Feature layer service definition.')
        else:
            print('Service definition updated successfully.')  